# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298569559727                   -0.85    5.4    144ms
  2   -8.300206622186       -2.79       -1.26    1.2    123ms
  3   -8.300436977259       -3.64       -1.89    3.2    102ms
  4   -8.300459468096       -4.65       -2.74    1.2   79.4ms
  5   -8.300463607898       -5.38       -3.04    2.4   97.1ms
  6   -8.300464117288       -6.29       -3.22    1.0   71.0ms
  7   -8.300464385415       -6.57       -3.37    1.0   77.7ms
  8   -8.300464519181       -6.87       -3.51    1.0   84.8ms
  9   -8.300464608359       -7.05       -3.71    1.1    105ms
 10   -8.300464630886       -7.65       -3.87    1.2   72.4ms
 11   -8.300464641983       -7.95       -4.20    1.1   67.5ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67003289822                   -0.70    6.9    357ms
  2   -16.67307571354       -2.52       -1.14    1.0    210ms
  3   -16.67356009242       -3.31       -1.87    2.4    217ms
  4   -16.67927087373       -2.24       -2.09    2.0    225ms
  5   -16.67928039986       -5.02       -2.12    1.0    202ms
  6   -16.67928485526       -5.35       -2.52    2.0    211ms
  7   -16.67928560946       -6.12       -3.00    2.6    215ms
  8   -16.67928574478       -6.87       -3.04    2.9    224ms
  9   -16.67928597513       -6.64       -3.28    1.0    170ms
 10   -16.67928609747       -6.91       -3.57    1.0    170ms
 11   -16.67928618146       -7.08       -3.84    1.8    205ms
 12   -16.67928621943       -7.42      

In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32801282858                   -0.56    6.6    1.28s
  2   -33.33462728547       -2.18       -1.00    1.2    661ms
  3   -33.33596264861       -2.87       -1.73    8.1    1.05s
  4   -33.33609990174       -3.86       -2.63    1.2    667ms
  5   -33.33679572990       -3.16       -2.23    9.6    1.16s
  6   -33.33682676300       -4.51       -2.29    1.1    646ms
  7   -33.33694217596       -3.94       -3.39    1.0    622ms
  8   -33.33694358947       -5.85       -3.59    3.8    885ms
  9   -33.33694366541       -7.12       -3.71    2.1    676ms
 10   -33.33694374015       -7.13       -4.02    1.5    633ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298602247270                   -0.85    5.8    225ms
  2   -8.300279902533       -2.78       -1.59    1.0   64.5ms
  3   -8.300419117183       -3.86       -2.49    2.1   72.7ms
  4   -8.300356058373   +   -4.20       -2.23    4.4    113ms
  5   -8.300464065388       -3.97       -3.41    1.0   67.1ms
  6   -8.300464525333       -6.34       -3.70    6.4    168ms
  7   -8.300464637066       -6.95       -4.16    1.1   73.0ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32783272283                   -0.56    7.5    1.31s
  2   -33.31959593592   +   -2.08       -1.27    1.2    577ms
  3   -17.13735635006   +    1.21       -0.46    6.2    1.25s
  4   -33.32764033187        1.21       -1.93    6.6    1.15s
  5   -33.05765518788   +   -0.57       -1.30    4.2    1.04s
  6   -32.81825081173   +   -0.62       -1.20    4.5    985ms
  7   -33.32653347939       -0.29       -1.80    4.1    918ms
  8   -33.33518132616       -2.06       -2.35    1.9    671ms
  9   -33.33638732665       -2.92       -2.45    2.6    848ms
 10   -33.33655341734       -3.78       -2.71    1.8    647ms
 11   -33.33684497013       -3.54       -3.00    2.5    651ms
 12   -33.33693327745       -4.05      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.